In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
# read the data

users_data = pd.read_pickle("../data/users_data_final.pkl")

In [3]:
# preview the data

users_data.head()

,user_id,number_transactions,total_amount_usd,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,term_deposit,date_joined,device
0,9231c446-cb16-4b2b-a7f7-ddfc8b25aaf6,3.0,2143.0,58,management,married,tertiary,False,True,False,None,261,1,-1,0,None,False,1998-08-23,mobile
1,bb92765a-08de-4963-b432-496524b39157,NaN,NaN,44,technician,single,secondary,False,True,False,None,151,1,-1,0,None,False,2008-07-15,desktop
2,573de577-49ef-42b9-83da-d3cfb817b5c1,2.0,2.0,33,entrepreneur,married,secondary,False,True,True,None,76,1,-1,0,None,False,2002-06-04,mobile
3,d6b66b9d-7c8f-4257-a682-e136f640b7e3,NaN,NaN,47,blue-collar,married,None,False,True,False,None,92,1,-1,0,None,False,1995-06-29,tablet
4,fade0b20-7594-4d9a-84cd-c02f79b1b526,1.0,1.0,33,None,single,None,False,False,False,None,198,1,-1,0,None,False,1995-08-01,mobile


In [4]:
# check the shape

users_data.shape

(45209, 19)

In [5]:
# check the structure

users_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45215
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   user_id              45209 non-null  object        
 1   number_transactions  35210 non-null  float64       
 2   total_amount_usd     35210 non-null  float64       
 3   age                  45209 non-null  int64         
 4   job                  44921 non-null  object        
 5   marital              45209 non-null  object        
 6   education            43352 non-null  object        
 7   default              45209 non-null  bool          
 8   housing              45209 non-null  bool          
 9   loan                 45209 non-null  bool          
 10  contact              32191 non-null  object        
 11  duration             45209 non-null  int64         
 12  campaign             45209 non-null  int64         
 13  pdays                45209 non-

In [6]:
# start by dropping user_id and date_joined
# user_id is not a useful features
# date joined needs work to be transformed and for now we will drop

users_data_cleaned = users_data.drop(['date_joined', 'user_id'], axis = 1)


In [7]:
list(users_data_cleaned)

['number_transactions',
 'total_amount_usd',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'term_deposit',
 'device']

In [8]:
# identify correlated features

users_data_cleaned.corr()

,number_transactions,total_amount_usd,age,default,housing,loan,duration,campaign,pdays,previous,term_deposit
number_transactions,1.000000,-0.163409,-0.008813,0.138838,0.030429,0.075319,-0.017220,0.026431,-0.030751,-0.023046,-0.053390
total_amount_usd,-0.163409,1.000000,0.095839,-0.065390,-0.066857,-0.084526,0.022586,-0.017274,0.006435,0.016952,0.050785
age,-0.008813,0.095839,1.000000,-0.017875,-0.185552,-0.015641,-0.004645,0.004767,-0.023745,0.001297,0.025168
default,0.138838,-0.065390,-0.017875,1.000000,-0.006020,0.077232,-0.010017,0.016819,-0.029982,-0.018331,-0.022421
housing,0.030429,-0.066857,-0.185552,-0.006020,1.000000,0.041341,0.005041,-0.023583,0.124197,0.037087,-0.139161
loan,0.075319,-0.084526,-0.015641,0.077232,0.041341,1.000000,-0.012395,0.009972,-0.022762,-0.011048,-0.068193
duration,-0.017220,0.022586,-0.004645,-0.010017,0.005041,-0.012395,1.000000,-0.084551,-0.001549,0.001213,0.394549
campaign,0.026431,-0.017274,0.004767,0.016819,-0.023583,0.009972,-0.084551,1.000000,-0.088636,-0.032860,-0.073179
pdays,-0.030751,0.006435,-0.023745,-0.029982,0.124197,-0.022762,-0.001549,-0.088636,1.000000,0.454817,0.103616
previous,-0.023046,0.016952,0.001297,-0.018331,0.037087,-0.011048,0.001213,-0.032860,0.454817,1.000000,0.093232


In [9]:
# check if any features have low standard deviation i.e std < 1

users_data_cleaned.describe()


,number_transactions,total_amount_usd,age,duration,campaign,pdays,previous
count,35210.000000,35210.000000,45209.000000,45209.000000,45209.000000,45209.000000,45209.000000
mean,3.977052,1369.417751,40.935853,258.153067,2.763897,40.199651,0.580349
std,3.814329,3063.412688,10.618653,257.525446,3.098076,100.130586,2.303489
min,1.000000,-8019.000000,18.000000,0.000000,1.000000,-1.000000,0.000000
25%,2.000000,73.000000,33.000000,103.000000,1.000000,-1.000000,0.000000
50%,3.000000,451.000000,39.000000,180.000000,2.000000,-1.000000,0.000000
75%,4.000000,1438.000000,48.000000,319.000000,3.000000,-1.000000,0.000000
max,20.000000,102127.000000,95.000000,4918.000000,63.000000,871.000000,275.000000


In [10]:
# handling missing data

users_data_cleaned.isnull().sum()


number_transactions     9999
total_amount_usd        9999
age                        0
job                      288
marital                    0
education               1857
default                    0
housing                    0
loan                       0
contact                13018
duration                   0
campaign                   0
pdays                      0
previous                   0
poutcome               36957
term_deposit               0
device                    94
dtype: int64

In [11]:
def identify_missing_data(df):
    """
    This function is used to identify missing data
    
    @param df pandas DataFrame
    
    @return a DataFrame with the percentage of missing data for every feature and the data types
    """
    
    percent_missing = df.isnull().mean()
    
    missing_value_df = pd.DataFrame(percent_missing).reset_index() # convert to DataFrame
    missing_value_df = missing_value_df.rename(columns = {"index" : "feature",
                                                                0 : "percent_missing"}) # rename columns

    missing_value_df = missing_value_df.sort_values(by = ['percent_missing'], ascending = False) # sort the values
    
    data_types_df = pd.DataFrame(df.dtypes).reset_index().rename(columns = {"index" : "feature",
                                                                0 : "data_type"}) # rename columns
    
    missing_value_df = missing_value_df.merge(data_types_df, on = "feature") # join the dataframe with datatype
    
    missing_value_df.percent_missing = round(missing_value_df.percent_missing*100, 2) # format the percent_missing
    
    return missing_value_df[missing_value_df.percent_missing > 0]

In [12]:
identify_missing_data(users_data_cleaned)

,feature,percent_missing,data_type
0,poutcome,81.75,object
1,contact,28.80,object
2,number_transactions,22.12,float64
3,total_amount_usd,22.12,float64
4,education,4.11,object
5,job,0.64,object
6,device,0.21,object


In [13]:
# drop features that have more than 60% missing data

users_data_cleaned.drop(['poutcome'], axis = 1, inplace = True)

In [14]:
identify_missing_data(users_data_cleaned)

,feature,percent_missing,data_type
0,contact,28.80,object
1,number_transactions,22.12,float64
2,total_amount_usd,22.12,float64
3,education,4.11,object
4,job,0.64,object
5,device,0.21,object


In [15]:
# which features have data type object

missing_df = identify_missing_data(users_data_cleaned)
non_numerical_var = list(missing_df[missing_df.data_type == "object"].feature)
non_numerical_var

['contact', 'education', 'job', 'device']

In [16]:
users_data_cleaned.contact.value_counts(dropna = False)

cellular     29285
NaN          13018
telephone     2906
Name: contact, dtype: int64

In [17]:
# replace the missing values with Unknown

users_data_cleaned.contact.fillna("Unknown", inplace = True)


In [18]:
users_data_cleaned.contact.value_counts(dropna = False)

cellular     29285
Unknown      13018
telephone     2906
Name: contact, dtype: int64

In [19]:
# replace non numerical variables , where missing values, with Unknown

users_data_cleaned.education.fillna("Unknown", inplace = True)
users_data_cleaned.job.fillna("Unknown", inplace = True)
users_data_cleaned.device.fillna("Unknown", inplace = True)



In [20]:
identify_missing_data(users_data_cleaned)

,feature,percent_missing,data_type
0,number_transactions,22.12,float64
1,total_amount_usd,22.12,float64


In [21]:
# replace numerical variables, where missing values, with zero

users_data_cleaned.number_transactions.fillna(0, inplace = True)
users_data_cleaned.total_amount_usd.fillna(0, inplace = True)


In [22]:
identify_missing_data(users_data_cleaned)

# WE SHOULD HAVE NO MISSING DATA!!!!!!!!! YAY!!!!

,feature,percent_missing,data_type


In [23]:
# Encoding the data

users_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45215
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   number_transactions  45209 non-null  float64
 1   total_amount_usd     45209 non-null  float64
 2   age                  45209 non-null  int64  
 3   job                  45209 non-null  object 
 4   marital              45209 non-null  object 
 5   education            45209 non-null  object 
 6   default              45209 non-null  bool   
 7   housing              45209 non-null  bool   
 8   loan                 45209 non-null  bool   
 9   contact              45209 non-null  object 
 10  duration             45209 non-null  int64  
 11  campaign             45209 non-null  int64  
 12  pdays                45209 non-null  int64  
 13  previous             45209 non-null  int64  
 14  term_deposit         45209 non-null  bool   
 15  device               45209 non-null 

In [24]:
users_data_cleaned.job.value_counts()

blue-collar      9731
management       9457
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
Unknown           288
Name: job, dtype: int64

In [25]:
# number of categories per variable with data type object

categories = (users_data_cleaned.select_dtypes("object")
            .apply(pd.Series.nunique)
            .sort_values(ascending=False))
        
categories

job          12
education     4
device        4
marital       3
contact       3
dtype: int64

In [26]:
# drop variables with more than 10 categories

users_data_cleaned.drop(['job'], axis = 1, inplace = True)

In [27]:
# check if dropped

list(users_data_cleaned)

['number_transactions',
 'total_amount_usd',
 'age',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'term_deposit',
 'device']

In [28]:
users_data_cleaned.marital

0         married
1          single
2         married
3         married
4          single
           ...   
45211     married
45212    divorced
45213     married
45214     married
45215     married
Name: marital, Length: 45209, dtype: object

In [29]:
pd.get_dummies(users_data_cleaned.marital, prefix='marital')

,marital_divorced,marital_married,marital_single
0,0,1,0
1,0,0,1
2,0,1,0
3,0,1,0
4,0,0,1
...,...,...,...
45211,0,1,0
45212,1,0,0
45213,0,1,0
45214,0,1,0


In [30]:
def one_hot(df, categorical_cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    
    for c in categorical_cols:
        dummies = pd.get_dummies(df[c], prefix=c)
        df = pd.concat([df, dummies], axis=1)
        df.drop(c, axis = 1, inplace = True)
    
    return df

In [31]:
# identify all categorical columns

# categorical_columns = ['marital', 'education', 'device', 'contact']
categorical_columns = list(users_data_cleaned.select_dtypes(['object']).columns)

# encode all of the categorical columns

users_data_cleaned_w_encoding = one_hot(users_data_cleaned, categorical_columns)

print(users_data_cleaned.shape)
print(users_data_cleaned_w_encoding.shape)

(45209, 15)
(45209, 25)


In [32]:
users_data_cleaned_w_encoding.head()

,number_transactions,total_amount_usd,age,default,housing,loan,duration,campaign,pdays,previous,...,education_primary,education_secondary,education_tertiary,contact_Unknown,contact_cellular,contact_telephone,device_Unknown,device_desktop,device_mobile,device_tablet
0,3.0,2143.0,58,False,True,False,261,1,-1,0,...,0,0,1,1,0,0,0,0,1,0
1,0.0,0.0,44,False,True,False,151,1,-1,0,...,0,1,0,1,0,0,0,1,0,0
2,2.0,2.0,33,False,True,True,76,1,-1,0,...,0,1,0,1,0,0,0,0,1,0
3,0.0,0.0,47,False,True,False,92,1,-1,0,...,0,0,0,1,0,0,0,0,0,1
4,1.0,1.0,33,False,False,False,198,1,-1,0,...,0,0,0,1,0,0,0,0,1,0


In [33]:
users_data_cleaned_w_encoding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45209 entries, 0 to 45215
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   number_transactions  45209 non-null  float64
 1   total_amount_usd     45209 non-null  float64
 2   age                  45209 non-null  int64  
 3   default              45209 non-null  bool   
 4   housing              45209 non-null  bool   
 5   loan                 45209 non-null  bool   
 6   duration             45209 non-null  int64  
 7   campaign             45209 non-null  int64  
 8   pdays                45209 non-null  int64  
 9   previous             45209 non-null  int64  
 10  term_deposit         45209 non-null  bool   
 11  marital_divorced     45209 non-null  uint8  
 12  marital_married      45209 non-null  uint8  
 13  marital_single       45209 non-null  uint8  
 14  education_Unknown    45209 non-null  uint8  
 15  education_primary    45209 non-null 

In [34]:
users_data_cleaned_w_encoding.to_pickle("../data/users_data_cleaned.pkl")